In [1]:
# This notebook generates the train and test dataset,  
# which user_vocab_id and movie_vocab_id are sorted by user and movie frequency

In [4]:
import os
import csv

In [5]:
# mapping old uid to new uid sorted by old uid frequency in train_data.dat

def mapping_id_by_sorted_freq(src_doc, mapping_doc, feature_idx, mapping):
    freq_cnt = {}
    with open('../data/train_data.dat', 'r') as file:
        rows = csv.reader(file,delimiter=':')
        for row in rows:
            old_vocab_id = row[feature_idx]
            freq_cnt[old_vocab_id] = freq_cnt.get(old_vocab_id, 0) + 1
    freq_cnt = [(old_vocab_id, cnt) for old_vocab_id, cnt in freq_cnt.items()]
    freq_cnt.sort(key = lambda x: x[1], reverse = True)
    for new_vocab_id, item in enumerate(freq_cnt):
        old_vocab_id, cnt = item
        mapping[old_vocab_id] = str(new_vocab_id+1)

    open(mapping_doc, 'w').close()
    outF = open(mapping_doc, 'w')
    with open(src_doc, 'r') as file:
        rows = csv.reader(file, delimiter='^')
        for row in rows:
            old_vocab_id = row[0]
            if old_vocab_id in mapping:
                new_row = [old_vocab_id, mapping[old_vocab_id]]
                new_row.extend(row[1:])
                new_row = ':'.join(new_row)
                outF.write(new_row)
                outF.write("\n")
        outF.close()
        

In [6]:
# replace old vocab id to new vocab id (popularity hashing id)。

def replace_old_vocab_id_with_popularity_hashing_id(source_doc, dest_doc, feature_idxs, mappings): 
    open(dest_doc, 'w').close()

    assert len(feature_idxs) == len(mappings)
    outF = open(dest_doc,'w')
    with open(source_doc, 'r') as file:
        rows = csv.reader(file,delimiter=':')
        for row in rows:
            for feature_idx, mapping in zip(feature_idxs, mappings):
                old_vocab_id = row[feature_idx]
                row = row[:feature_idx] + [mapping[old_vocab_id]] + row[feature_idx+1:]
            outF.write(":".join(row) + '\n')
        outF.close()

In [7]:
user_replace_doc_source = 'train_data.dat'
movie_replace_doc_source = 'data_sorted_user.dat'
resorted_data_doc = 'data_sorted.dat'
user_mapping, movie_mapping = {}, {}
mapping_id_by_sorted_freq('../data/users.dat', '../data/user_id_mapping.dat', 0, user_mapping)
mapping_id_by_sorted_freq('../data/movies.dat', '../data/movie_id_mapping.dat', 1, movie_mapping)
replace_old_vocab_id_with_popularity_hashing_id('../data/train_data.dat', '../data/train_data_sorted_vocab_id.dat', [0, 1], [user_mapping, movie_mapping])
replace_old_vocab_id_with_popularity_hashing_id('../data/test_data.dat', '../data/test_data_sorted_vocab_id.dat', [0, 1], [user_mapping, movie_mapping])


In [8]:
# write metrics to train_meta_data.dat
######metadata.dat########
# num_user:69280
# num_movie:36365
# num_category:28
######metadata.dat########
num_user, num_movie, num_category = 0, 0, 28
with open('../data/user_id_mapping.dat', 'r') as f:
    for line in f:
        num_user += 1
with open('../data/movie_id_mapping.dat', 'r') as f:
    for line in f:
        num_movie += 1
print(num_user, num_movie)
        
keys = ['num_user', 'num_movie', 'num_category']
values = [num_user, num_movie, num_category]
open('../data/train_meta_data.dat', 'w').close()
with open("../data/train_meta_data.dat", "w") as file:
    for i in range(3):
        file.write(keys[i] + ":" + str(values[i]) + "\n")

39310 32158


In [ ]:
#mapping movie new vocab id to movie name
